In [1]:
%matplotlib inline

In [2]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions (precision=4, linewidth=100)
from matplotlib import pyplot as plt
from os.path import *
import pprint

In [3]:
import utils; reload (utils)
from utils import plots

Using Theano backend.


In [4]:
batch_size=2

base_dir = os.getcwd()
data_dir = join (base_dir, "data")
data_dir = join (data_dir, "sample")
test_dir = join (data_dir, "test")
train_dir = join (data_dir, "train")
valid_dir = join (data_dir, "valid")

In [5]:
import vgg16; reload (vgg16)
from vgg16 import Vgg16

In [6]:
vgg = Vgg16 ()
batches = vgg.get_batches (train_dir, batch_size=batch_size)
val_batches = vgg.get_batches (valid_dir, batch_size=batch_size*2)
vgg.finetune (batches)
vgg.fit (batches, val_batches, nb_epoch=1)

Found 10 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/1
10/10 [==============================] - 15s - loss: 1.4312 - acc: 0.2000 - val_loss: 1.4014 - val_acc: 0.5000


In [7]:
print (test_dir)

C:\cygwin64\home\sukruthp\Documents\fastai\mycode\fastai_1\data\sample\test


In [8]:
pp = pprint.PrettyPrinter (indent=4)
pp.pprint (vgg.test(test_dir, batch_size))

Found 5 images belonging to 1 classes.
(   <keras.preprocessing.image.DirectoryIterator object at 0x000000000F6F5F60>,
    array([[  9.9995e-01,   5.3195e-05],
       [  9.9993e-01,   6.7181e-05],
       [  9.8514e-01,   1.4858e-02],
       [  8.9716e-01,   1.0284e-01],
       [  7.5509e-01,   2.4491e-01]], dtype=float32))


In [9]:
gen, result = vgg.test (test_dir, batch_size)

Found 5 images belonging to 1 classes.


In [10]:
pp.pprint (gen)

In [11]:
print (vars(gen))

{'batch_index': 1, 'shuffle': False, 'index_generator': <generator object _flow_index at 0x000000001A7CB678>, 'lock': <thread.lock object at 0x0000000012102430>, 'save_format': 'jpeg', 'dim_ordering': 'th', 'nb_sample': 5, 'save_to_dir': None, 'color_mode': 'rgb', 'image_shape': (3, 224, 224), 'target_size': (224, 224), 'save_prefix': '', 'image_data_generator': <keras.preprocessing.image.ImageDataGenerator object at 0x000000000F6F5C50>, 'class_mode': None, 'class_indices': {'unknown': 0}, 'total_batches_seen': 13, 'batch_size': 2, 'filenames': ['unknown\\1978.jpg', 'unknown\\3465.jpg', 'unknown\\3946.jpg', 'unknown\\5819.jpg', 'unknown\\7228.jpg'], 'nb_class': 1, 'n': 5, 'classes': array([0, 0, 0, 0, 0]), 'directory': 'C:\\cygwin64\\home\\sukruthp\\Documents\\fastai\\mycode\\fastai_1\\data\\sample\\test'}


In [20]:
test_batch = vgg.get_batches (test_dir, batch_size=batch_size, class_mode=None)

Found 5 images belonging to 1 classes.


In [23]:
def pred_batch (imgs):
    preds = vgg.model.predict (imgs)
    idxs = np.argmax (preds, axis=1)
    
    print ('Shape: {}'.format (preds.shape))
    print 

SyntaxError: unexpected EOF while parsing (<ipython-input-23-066433eb5f82>, line 1)